# 🎯 Segmentação RFM (Recency, Frequency, Monetary) - Olist E-Commerce

**Autor:** André Bomfim  
**Data:** Novembro 2024  
**Objetivo:** Segmentar clientes usando análise RFM para identificar Champions, clientes em risco e oportunidades de conversão com recomendações personalizadas

---

## 📋 Índice

1. [Setup e Configuração](#1-setup)
2. [Extração e Cálculo RFM](#2-extracao)
3. [Scores RFM](#3-scores)
4. [Segmentação por Regras](#4-segmentacao)
5. [Análise por Segmento](#5-analise)
6. [Visualizações Avançadas](#6-visualizacoes)
7. [Análise Geográfica](#7-geografica)
8. [Recomendações e Ações](#8-acoes)

## 1. Setup e Configuração {#1-setup}

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Carregar variáveis de ambiente
load_dotenv()

# Configuração BigQuery
PROJECT_ID = os.getenv('GCP_PROJECT_ID')
DATASET_ID = os.getenv('GCP_DATASET_ID', 'olist_ecommerce')

# Cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Criar diretório para imagens
os.makedirs('../docs/images', exist_ok=True)

print(f"✓ Setup completo - Projeto: {PROJECT_ID}, Dataset: {DATASET_ID}")

In [ ]:
def query_bigquery(query: str) -> pd.DataFrame:
    """Helper para executar queries no BigQuery"""
    return client.query(query).to_dataframe()

def save_plot(fig, filename):
    """Salvar gráfico com tratamento de erro"""
    try:
        fig.savefig(f'../docs/images/{filename}', dpi=300, bbox_inches='tight')
        print(f"✓ Gráfico salvo: docs/images/{filename}")
    except Exception as e:
        print(f"⚠️ Erro ao salvar gráfico: {e}")

## 2. Extração e Cálculo RFM {#2-extracao}

In [ ]:
# Query RFM Base
query_rfm_base = f"""
WITH customer_orders AS (
    SELECT 
        c.customer_unique_id,
        c.customer_state,
        COUNT(DISTINCT o.order_id) as frequency,
        SUM(p.payment_value) as monetary,
        MAX(o.order_purchase_timestamp) as last_purchase_date,
        MIN(o.order_purchase_timestamp) as first_purchase_date,
        AVG(p.payment_value) as avg_order_value
    FROM `{PROJECT_ID}.{DATASET_ID}.orders` o
    JOIN `{PROJECT_ID}.{DATASET_ID}.customers` c ON o.customer_id = c.customer_id
    JOIN `{PROJECT_ID}.{DATASET_ID}.payments` p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_unique_id, c.customer_state
)
SELECT
    customer_unique_id,
    customer_state,
    frequency,
    monetary,
    last_purchase_date,
    first_purchase_date,
    avg_order_value,
    DATE_DIFF(DATE('2018-10-31'), DATE(last_purchase_date), DAY) as recency
FROM customer_orders
WHERE frequency > 0 AND monetary > 0
ORDER BY monetary DESC
"""

df_rfm = query_bigquery(query_rfm_base)
print(f"✓ {len(df_rfm):,} clientes carregados com métricas RFM")
print(f"✓ Período: {df_rfm['first_purchase_date'].min()} a {df_rfm['last_purchase_date'].max()}")
print(f"✓ Estados analisados: {df_rfm['customer_state'].nunique()}")

In [ ]:
# Estatísticas Descritivas RFM
print("="*60)
print("ESTATÍSTICAS DESCRITIVAS - RFM BASE")
print("="*60)

print(f"\n📊 MÉTRICAS GERAIS:")
print(f"  • Clientes únicos: {len(df_rfm):,}")
print(f"  • Receita total: R$ {df_rfm['monetary'].sum():,.2f}")
print(f"  • Ticket médio: R$ {df_rfm['avg_order_value'].mean():.2f}")

print(f"\n🎯 RECENCY (dias desde última compra):")
print(f"  • Média: {df_rfm['recency'].mean():.1f} dias")
print(f"  • Mediana: {df_rfm['recency'].median():.1f} dias")
print(f"  • P25: {df_rfm['recency'].quantile(0.25):.1f} dias")
print(f"  • P75: {df_rfm['recency'].quantile(0.75):.1f} dias")

print(f"\n🔄 FREQUENCY (número de pedidos):")
print(f"  • Média: {df_rfm['frequency'].mean():.2f} pedidos")
print(f"  • Mediana: {df_rfm['frequency'].median():.0f} pedidos")
print(f"  • Clientes com 2+ pedidos: {(df_rfm['frequency'] >= 2).sum():,} ({(df_rfm['frequency'] >= 2).sum()/len(df_rfm)*100:.1f}%)")

print(f"\n💰 MONETARY (valor total gasto):")
print(f"  • Média: R$ {df_rfm['monetary'].mean():.2f}")
print(f"  • Mediana: R$ {df_rfm['monetary'].median():.2f}")
print(f"  • P90: R$ {df_rfm['monetary'].quantile(0.90):.2f}")
print(f"  • P95: R$ {df_rfm['monetary'].quantile(0.95):.2f}")

In [ ]:
# Visualização Distribuição RFM
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histograma Recency
axes[0, 0].hist(df_rfm['recency'], bins=50, edgecolor='black', alpha=0.7, color='red')
axes[0, 0].set_xlabel('Recency (dias desde última compra)')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].set_title('Distribuição de Recency')
axes[0, 0].axvline(df_rfm['recency'].mean(), color='darkred', linestyle='--', label='Média')
axes[0, 0].legend()

# Histograma Frequency
axes[0, 1].hist(df_rfm['frequency'], bins=30, edgecolor='black', alpha=0.7, color='blue')
axes[0, 1].set_xlabel('Frequency (número de pedidos)')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].set_title('Distribuição de Frequency')
axes[0, 1].set_xlim(0, 10)  # Zoom para melhor visualização

# Histograma Monetary (log scale)
axes[1, 0].hist(df_rfm['monetary'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1, 0].set_xlabel('Monetary (valor total gasto - R$)')
axes[1, 0].set_ylabel('Frequência')
axes[1, 0].set_title('Distribuição de Monetary')
axes[1, 0].set_xlim(0, 500)  # Zoom para melhor visualização

# Scatter R vs F (colorido por M)
scatter = axes[1, 1].scatter(df_rfm['recency'], df_rfm['frequency'], 
                            c=df_rfm['monetary'], cmap='viridis', alpha=0.6, s=20)
axes[1, 1].set_xlabel('Recency (dias)')
axes[1, 1].set_ylabel('Frequency (pedidos)')
axes[1, 1].set_title('Recency vs Frequency (colorido por Monetary)')
plt.colorbar(scatter, ax=axes[1, 1], label='Monetary (R$)')

plt.tight_layout()
save_plot(fig, 'rfm_distribution.png')
plt.show()

## 3. Scores RFM {#3-scores}

In [ ]:
# Calcular Scores RFM usando quintis
print("🎯 CALCULANDO SCORES RFM...")

# R_score: invertido (menor recency = score maior)
df_rfm['R_score'] = pd.qcut(df_rfm['recency'], q=5, labels=[5, 4, 3, 2, 1], duplicates='drop')

# F_score: direto (maior frequency = score maior)
df_rfm['F_score'] = pd.qcut(df_rfm['frequency'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# M_score: direto (maior monetary = score maior)
df_rfm['M_score'] = pd.qcut(df_rfm['monetary'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')

# Converter para numérico
df_rfm['R_score'] = df_rfm['R_score'].astype(int)
df_rfm['F_score'] = df_rfm['F_score'].astype(int)
df_rfm['M_score'] = df_rfm['M_score'].astype(int)

# RFM Score (string)
df_rfm['RFM_score'] = df_rfm['R_score'].astype(str) + df_rfm['F_score'].astype(str) + df_rfm['M_score'].astype(str)

# RFM Score Numérico (média ponderada)
df_rfm['RFM_score_numeric'] = (df_rfm['R_score'] * 0.4 + 
                              df_rfm['F_score'] * 0.3 + 
                              df_rfm['M_score'] * 0.3)

print("✓ Scores RFM calculados com sucesso!")
print(f"\n📊 DISTRIBUIÇÃO DOS SCORES:")
print(f"  • R_score: {df_rfm['R_score'].value_counts().sort_index().to_dict()}")
print(f"  • F_score: {df_rfm['F_score'].value_counts().sort_index().to_dict()}")
print(f"  • M_score: {df_rfm['M_score'].value_counts().sort_index().to_dict()}")

print(f"\n🏆 TOP 5 RFM SCORES:")
top_rfm_scores = df_rfm['RFM_score'].value_counts().head()
for score, count in top_rfm_scores.items():
    print(f"  • {score}: {count:,} clientes ({count/len(df_rfm)*100:.1f}%)")

In [ ]:
# Visualização Distribuição Scores RFM
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribuição R_score
r_counts = df_rfm['R_score'].value_counts().sort_index()
axes[0, 0].bar(r_counts.index, r_counts.values, color='red', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('R Score')
axes[0, 0].set_ylabel('Número de Clientes')
axes[0, 0].set_title('Distribuição do R Score (Recency)')
axes[0, 0].set_xticks([1, 2, 3, 4, 5])

# Distribuição F_score
f_counts = df_rfm['F_score'].value_counts().sort_index()
axes[0, 1].bar(f_counts.index, f_counts.values, color='blue', alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('F Score')
axes[0, 1].set_ylabel('Número de Clientes')
axes[0, 1].set_title('Distribuição do F Score (Frequency)')
axes[0, 1].set_xticks([1, 2, 3, 4, 5])

# Distribuição M_score
m_counts = df_rfm['M_score'].value_counts().sort_index()
axes[1, 0].bar(m_counts.index, m_counts.values, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('M Score')
axes[1, 0].set_ylabel('Número de Clientes')
axes[1, 0].set_title('Distribuição do M Score (Monetary)')
axes[1, 0].set_xticks([1, 2, 3, 4, 5])

# Histograma RFM_score_numeric
axes[1, 1].hist(df_rfm['RFM_score_numeric'], bins=20, edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].set_xlabel('RFM Score Numérico')
axes[1, 1].set_ylabel('Frequência')
axes[1, 1].set_title('Distribuição do RFM Score Numérico')
axes[1, 1].axvline(df_rfm['RFM_score_numeric'].mean(), color='darkred', linestyle='--', label='Média')
axes[1, 1].legend()

plt.tight_layout()
save_plot(fig, 'rfm_scores_distribution.png')
plt.show()

## 4. Segmentação por Regras {#4-segmentacao}

In [ ]:
# Função de Segmentação RFM
def assign_segment(row):
    r, f, m = row['R_score'], row['F_score'], row['M_score']
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif f >= 4:
        return 'Loyal Customers'
    elif r >= 4 and f >= 2 and m >= 2:
        return 'Potential Loyalist'
    elif r >= 4 and f == 1:
        return 'New Customers'
    elif r >= 3 and f == 1 and m >= 2:
        return 'Promising'
    elif r >= 2 and f >= 2 and m >= 2:
        return 'Need Attention'
    elif r >= 2 and f <= 2 and m <= 2:
        return 'About To Sleep'
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    elif r <= 2 and f >= 4 and m >= 4:
        return 'Cannot Lose Them'
    elif r <= 2 and f <= 2 and m <= 2:
        return 'Hibernating'
    elif r == 1:
        return 'Lost'
    else:
        return 'Others'

# Aplicar segmentação
df_rfm['segment'] = df_rfm.apply(assign_segment, axis=1)

# Mapa de Prioridades
PRIORITY_MAP = {
    'Champions': 1,
    'Cannot Lose Them': 1,
    'Loyal Customers': 2,
    'At Risk': 2,
    'Potential Loyalist': 3,
    'Need Attention': 3,
    'Promising': 4,
    'New Customers': 4,
    'About To Sleep': 3,
    'Hibernating': 5,
    'Lost': 6,
    'Others': 5
}

df_rfm['priority'] = df_rfm['segment'].map(PRIORITY_MAP)

print("✓ Segmentação RFM aplicada com sucesso!")
print(f"\n📊 DISTRIBUIÇÃO POR SEGMENTO:")
segment_counts = df_rfm['segment'].value_counts()
for segment, count in segment_counts.items():
    pct = count / len(df_rfm) * 100
    print(f"  • {segment}: {count:,} clientes ({pct:.1f}%)")

# ============ CORREÇÃO: QUALIDADE DA SEGMENTAÇÃO ============
print("\n📊 QUALIDADE DA SEGMENTAÇÃO:")
print(f"  • Cobertura: {(df_rfm['segment'] != 'Others').sum()/len(df_rfm)*100:.1f}% "
      f"({(df_rfm['segment'] == 'Others').sum()} clientes em 'Others')")
print(f"  • Concentração: Top 3 segmentos = "
      f"{segment_counts.head(3).sum()/len(df_rfm)*100:.1f}% dos clientes")

In [ ]:
# Visualização Segmentos RFM
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Preparar dados agregados por segmento
segment_summary = df_rfm.groupby('segment').agg({
    'customer_unique_id': 'count',
    'monetary': 'sum',
    'recency': 'mean',
    'frequency': 'mean',
    'RFM_score_numeric': 'mean'
}).round(2)

segment_summary = segment_summary.rename(columns={
    'customer_unique_id': 'customers',
    'monetary': 'revenue'
})

segment_summary['customer_pct'] = (segment_summary['customers'] / segment_summary['customers'].sum()) * 100
segment_summary['revenue_pct'] = (segment_summary['revenue'] / segment_summary['revenue'].sum()) * 100
segment_summary['revenue_per_customer'] = segment_summary['revenue'] / segment_summary['customers']

# Ordenar por revenue
segment_summary = segment_summary.sort_values('revenue', ascending=False)

# Gráfico 1: Distribuição de clientes por segmento
segment_customers = segment_summary.sort_values('customers', ascending=True)
axes[0, 0].barh(range(len(segment_customers)), segment_customers['customers'], 
               color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_yticks(range(len(segment_customers)))
axes[0, 0].set_yticklabels(segment_customers.index)
axes[0, 0].set_xlabel('Número de Clientes')
axes[0, 0].set_title('Distribuição de Clientes por Segmento')

# Gráfico 2: Distribuição de receita por segmento
segment_revenue = segment_summary.sort_values('revenue', ascending=True)
axes[0, 1].barh(range(len(segment_revenue)), segment_revenue['revenue']/1000, 
               color='lightgreen', alpha=0.7, edgecolor='black')
axes[0, 1].set_yticks(range(len(segment_revenue)))
axes[0, 1].set_yticklabels(segment_revenue.index)
axes[0, 1].set_xlabel('Receita (R$ mil)')
axes[0, 1].set_title('Distribuição de Receita por Segmento')

# Gráfico 3: Curva de Pareto
pareto_data = segment_summary.sort_values('revenue_pct', ascending=False)
pareto_data['cumulative_revenue_pct'] = pareto_data['revenue_pct'].cumsum()
pareto_data['cumulative_customer_pct'] = pareto_data['customer_pct'].cumsum()

ax2 = axes[1, 0]
ax2.plot(pareto_data['cumulative_customer_pct'], pareto_data['cumulative_revenue_pct'], 
         marker='o', linewidth=2, label='Curva de Pareto', color='blue')
ax2.plot([0, 100], [0, 100], '--', color='gray', alpha=0.7, label='Igualdade')
ax2.axhline(80, color='red', linestyle='--', alpha=0.7, label='80% Receita')
ax2.set_xlabel('% Acumulado de Clientes')
ax2.set_ylabel('% Acumulado de Receita')
ax2.set_title('Curva de Pareto: Clientes vs Receita')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 4: Top 5 segmentos por receita (pie chart)
top_5_segments = segment_summary.head(5)
axes[1, 1].pie(top_5_segments['revenue_pct'], labels=top_5_segments.index, 
               autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Top 5 Segmentos por Participação na Receita')

plt.tight_layout()
save_plot(fig, 'rfm_segments_overview.png')
plt.show()

## 5. Análise por Segmento {#5-analise}

In [ ]:
# Análise Detalhada por Segmento
print("="*80)
print("ANÁLISE DETALHADA POR SEGMENTO RFM")
print("="*80)

print(f"\n📊 RESUMO POR SEGMENTO:")
for segment in segment_summary.index:
    data = segment_summary.loc[segment]
    print(f"\n🎯 {segment.upper()}:")
    print(f"   • Clientes: {data['customers']:,} ({data['customer_pct']:.1f}%)")
    print(f"   • Receita: R$ {data['revenue']:,.2f} ({data['revenue_pct']:.1f}%)")
    print(f"   • Revenue per customer: R$ {data['revenue_per_customer']:.2f}")
    print(f"   • Recency médio: {data['recency']:.1f} dias")
    print(f"   • Frequency médio: {data['frequency']:.2f} pedidos")

# Insights Principais
print(f"\n💡 INSIGHTS PRINCIPAIS:")
champions_revenue_pct = segment_summary.loc['Champions', 'revenue_pct'] if 'Champions' in segment_summary.index else 0
top_20_revenue = segment_summary.head(4)['revenue_pct'].sum()
new_customers_pct = segment_summary.loc['New Customers', 'customer_pct'] if 'New Customers' in segment_summary.index else 0

print(f"  • Champions geram {champions_revenue_pct:.1f}% da receita")
print(f"  • Top 4 segmentos: {top_20_revenue:.1f}% da receita (Princípio 80/20)")
print(f"  • New Customers: {new_customers_pct:.1f}% do total (oportunidade de conversão)")
print(f"  • Segmentos At Risk + Cannot Lose Them: {(segment_summary.loc['At Risk', 'revenue_pct'] if 'At Risk' in segment_summary.index else 0) + (segment_summary.loc['Cannot Lose Them', 'revenue_pct'] if 'Cannot Lose Them' in segment_summary.index else 0):.1f}% da receita em risco")

# ============ CORREÇÃO: TABELAS DE SAÍDA ============
print("\n📊 TABELAS DE SAÍDA:")

# Tabela 1: df_rfm completo
print("\n1️⃣ df_rfm (primeiras 5 linhas):")
print(df_rfm[['customer_unique_id', 'customer_state', 'recency', 
              'frequency', 'monetary', 'R_score', 'F_score', 'M_score', 
              'RFM_score', 'segment']].head())

# Tabela 2: segment_summary
print("\n2️⃣ segment_summary:")
print(segment_summary[['customers', 'customer_pct', 'revenue', 
                       'revenue_pct', 'revenue_per_customer']])

# Tabela 3: action_map
ACTION_MAP = {
    'Champions': "🏆 RECOMPENSE! Programa VIP, early access, benefícios exclusivos, pedir referrals",
    'Loyal Customers': "⭐ UPSELL & CROSS-SELL! Produtos premium, pedir reviews, programa fidelidade",
    'Potential Loyalist': "📈 NURTURING! Email marketing personalizado, ofertas segmentadas",
    'New Customers': "👋 ONBOARDING! Sequência boas-vindas, cupom segunda compra, educar sobre marca",
    'Promising': "🌟 INCENTIVE! Cupom 15% segunda compra, mostrar produtos relacionados",
    'Need Attention': "🔔 REATIVE! Ofertas limitadas, lembre que sentem falta, pesquisa satisfação",
    'About To Sleep': "😴 ALERTA! Email re-engajamento, novidades, ofertas especiais",
    'At Risk': "⚠️ WIN-BACK! Cupom 20%, contato personalizado, pesquisa por que pararam",
    'Cannot Lose Them': "🚨 URGENTE! Oferta especial, contato direto, recuperação agressiva",
    'Hibernating': "💤 REATIVAÇÃO! Campanha massiva, cupom agressivo ou considerar não investir",
    'Lost': "❌ CUSTO ALTO! Avaliar ROI, possivelmente não investir recursos",
    'Others': "🔍 ANALISAR! Verificar padrões, possível subsegmentação"
}

action_df = pd.DataFrame({
    'segment': segment_summary.index,
    'acao_recomendada': [ACTION_MAP[seg] for seg in segment_summary.index],
    'prioridade': [PRIORITY_MAP[seg] for seg in segment_summary.index],
    'customers': segment_summary['customers'].values,
    'revenue_pct': segment_summary['revenue_pct'].values
})
print("\n3️⃣ action_map:")
print(action_df.sort_values('prioridade'))

In [ ]:
# Visualização Métricas por Segmento
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Ordenar segmentos por revenue para consistência
segment_ordered = segment_summary.sort_values('revenue', ascending=False)

# Gráfico 1: LTV médio por segmento
axes[0, 0].bar(range(len(segment_ordered)), segment_ordered['revenue_per_customer'], 
               color='gold', alpha=0.7, edgecolor='black')
axes[0, 0].set_xticks(range(len(segment_ordered)))
axes[0, 0].set_xticklabels(segment_ordered.index, rotation=45)
axes[0, 0].set_ylabel('LTV Médio (R$)')
axes[0, 0].set_title('LTV Médio por Segmento')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Gráfico 2: Frequency média por segmento
axes[0, 1].bar(range(len(segment_ordered)), segment_ordered['frequency'], 
               color='blue', alpha=0.7, edgecolor='black')
axes[0, 1].set_xticks(range(len(segment_ordered)))
axes[0, 1].set_xticklabels(segment_ordered.index, rotation=45)
axes[0, 1].set_ylabel('Frequency Média (pedidos)')
axes[0, 1].set_title('Frequency Média por Segmento')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Gráfico 3: Recency médio por segmento (invertido - menor é melhor)
axes[1, 0].bar(range(len(segment_ordered)), segment_ordered['recency'], 
               color='red', alpha=0.7, edgecolor='black')
axes[1, 0].set_xticks(range(len(segment_ordered)))
axes[1, 0].set_xticklabels(segment_ordered.index, rotation=45)
axes[1, 0].set_ylabel('Recency Médio (dias)')
axes[1, 0].set_title('Recency Médio por Segmento (menor = melhor)')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Gráfico 4: RFM Score médio por segmento
axes[1, 1].bar(range(len(segment_ordered)), segment_ordered['RFM_score_numeric'], 
               color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(range(len(segment_ordered)))
axes[1, 1].set_xticklabels(segment_ordered.index, rotation=45)
axes[1, 1].set_ylabel('RFM Score Médio')
axes[1, 1].set_title('RFM Score Médio por Segmento')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
save_plot(fig, 'rfm_segment_metrics.png')
plt.show()

## 6. Visualizações Avançadas {#6-visualizacoes}

In [ ]:
# Scatter Matrix RFM
plt.close('all')
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Amostrar dados para melhor visualização
sample_size = min(1000, len(df_rfm))
df_sample = df_rfm.sample(sample_size, random_state=42)

# Criar mapa de cores para segmentos
unique_segments = df_sample['segment'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(unique_segments)))
color_map = dict(zip(unique_segments, colors))

# Gráfico 1: R vs F colorido por segmento
for segment in unique_segments:
    segment_data = df_sample[df_sample['segment'] == segment]
    axes[0, 0].scatter(segment_data['recency'], segment_data['frequency'], 
                      c=[color_map[segment]], label=segment, alpha=0.7, s=50)
axes[0, 0].set_xlabel('Recency (dias)')
axes[0, 0].set_ylabel('Frequency (pedidos)')
axes[0, 0].set_title('Recency vs Frequency (colorido por Segmento)')
axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Gráfico 2: R vs M colorido por segmento
for segment in unique_segments:
    segment_data = df_sample[df_sample['segment'] == segment]
    axes[0, 1].scatter(segment_data['recency'], segment_data['monetary'], 
                      c=[color_map[segment]], label=segment, alpha=0.7, s=50)
axes[0, 1].set_xlabel('Recency (dias)')
axes[0, 1].set_ylabel('Monetary (R$)')
axes[0, 1].set_title('Recency vs Monetary (colorido por Segmento)')
axes[0, 1].set_ylim(0, 500)  # Zoom para melhor visualização

# Gráfico 3: F vs M colorido por segmento
for segment in unique_segments:
    segment_data = df_sample[df_sample['segment'] == segment]
    axes[1, 0].scatter(segment_data['frequency'], segment_data['monetary'], 
                      c=[color_map[segment]], label=segment, alpha=0.7, s=50)
axes[1, 0].set_xlabel('Frequency (pedidos)')
axes[1, 0].set_ylabel('Monetary (R$)')
axes[1, 0].set_title('Frequency vs Monetary (colorido por Segmento)')
axes[1, 0].set_xlim(0, 10)
axes[1, 0].set_ylim(0, 500)

# Gráfico 4: Heatmap de concentração RFM
# Criar pivot table para heatmap
rfm_pivot = df_rfm.pivot_table(
    index='R_score',
    columns='F_score',
    values='monetary',
    aggfunc='mean'
).fillna(0)

im = axes[1, 1].imshow(rfm_pivot, cmap='YlOrRd', aspect='auto')
axes[1, 1].set_xlabel('F Score')
axes[1, 1].set_ylabel('R Score')
axes[1, 1].set_title('Heatmap: Monetary Médio por R Score vs F Score')
axes[1, 1].set_xticks(range(len(rfm_pivot.columns)))
axes[1, 1].set_xticklabels(rfm_pivot.columns)
axes[1, 1].set_yticks(range(len(rfm_pivot.index)))
axes[1, 1].set_yticklabels(rfm_pivot.index)
plt.colorbar(im, ax=axes[1, 1], label='Monetary Médio (R$)')

# ============ CORREÇÃO: ANOTAR VALORES NO HEATMAP ============
for i in range(len(rfm_pivot.index)):
    for j in range(len(rfm_pivot.columns)):
        value = rfm_pivot.iloc[i, j]
        if value > 0:
            axes[1, 1].text(j, i, f'{value:.0f}', 
                          ha='center', va='center', 
                          color='white' if value > rfm_pivot.values.mean() else 'black',
                          fontsize=9)

plt.tight_layout()
save_plot(fig, 'rfm_scatter_matrix.png')
plt.show()

## 7. Análise Geográfica {#7-geografica}

In [ ]:
# Análise Geográfica por Segmento
print("🌎 ANÁLISE GEOGRÁFICA DOS SEGMENTOS RFM")

# Segmentos por estado
state_segment = df_rfm.groupby(['customer_state', 'segment']).agg({
    'customer_unique_id': 'count',
    'monetary': 'sum'
}).reset_index()

# Top estados por segmentos importantes
important_segments = ['Champions', 'Loyal Customers', 'Cannot Lose Them', 'At Risk']

print(f"\n🏆 CHAMPIONS POR ESTADO (Top 5):")
champions_by_state = state_segment[state_segment['segment'] == 'Champions']\
    .nlargest(5, 'customer_unique_id')[['customer_state', 'customer_unique_id', 'monetary']]
for _, row in champions_by_state.iterrows():
    print(f"  • {row['customer_state']}: {row['customer_unique_id']:,} champions (R$ {row['monetary']:,.2f})")

print(f"\n⚠️  CLIENTES EM RISCO POR ESTADO (Top 5):")
at_risk_by_state = state_segment[state_segment['segment'].isin(['At Risk', 'Cannot Lose Them'])]\
    .groupby('customer_state').agg({'customer_unique_id': 'sum', 'monetary': 'sum'})\
    .nlargest(5, 'customer_unique_id')
for state, row in at_risk_by_state.iterrows():
    print(f"  • {state}: {row['customer_unique_id']:,} clientes em risco (R$ {row['monetary']:,.2f})")

print(f"\n📈 CONCENTRAÇÃO GEOGRÁFICA:")
top_state_revenue = state_segment.groupby('customer_state')['monetary'].sum().nlargest(3)
total_revenue = state_segment['monetary'].sum()
print(f"  • Top 3 estados: {top_state_revenue.index.tolist()}")
print(f"  • Participação: {(top_state_revenue.sum() / total_revenue * 100):.1f}% da receita total")

## 8. Recomendações e Ações {#8-acoes}

In [ ]:
# Mapa de Ações por Segmento
ACTION_MAP = {
    'Champions': "🏆 RECOMPENSE! Programa VIP, early access, benefícios exclusivos, pedir referrals",
    'Loyal Customers': "⭐ UPSELL & CROSS-SELL! Produtos premium, pedir reviews, programa fidelidade",
    'Potential Loyalist': "📈 NURTURING! Email marketing personalizado, ofertas segmentadas",
    'New Customers': "👋 ONBOARDING! Sequência boas-vindas, cupom segunda compra, educar sobre marca",
    'Promising': "🌟 INCENTIVE! Cupom 15% segunda compra, mostrar produtos relacionados",
    'Need Attention': "🔔 REATIVE! Ofertas limitadas, lembre que sentem falta, pesquisa satisfação",
    'About To Sleep': "😴 ALERTA! Email re-engajamento, novidades, ofertas especiais",
    'At Risk': "⚠️ WIN-BACK! Cupom 20%, contato personalizado, pesquisa por que pararam",
    'Cannot Lose Them': "🚨 URGENTE! Oferta especial, contato direto, recuperação agressiva",
    'Hibernating': "💤 REATIVAÇÃO! Campanha massiva, cupom agressivo ou considerar não investir",
    'Lost': "❌ CUSTO ALTO! Avaliar ROI, possivelmente não investir recursos",
    'Others': "🔍 ANALISAR! Verificar padrões, possível subsegmentação"
}

# Sumário Executivo Final
print("\n" + "="*80)
print("🎯 RESUMO EXECUTIVO - SEGMENTAÇÃO RFM")
print("="*80)

# ============ CORREÇÃO: VALIDAÇÃO DE INSIGHTS ESPERADOS ============
print("\n🎯 VALIDAÇÃO DE INSIGHTS ESPERADOS:")
champions_customer_pct = segment_summary.loc['Champions', 'customer_pct'] if 'Champions' in segment_summary.index else 0
champions_revenue_pct = segment_summary.loc['Champions', 'revenue_pct'] if 'Champions' in segment_summary.index else 0
print(f"  ✅ Champions: {champions_customer_pct:.1f}% clientes, {champions_revenue_pct:.1f}% revenue")
print(f"     (Esperado: 5-10% clientes, 30-40% revenue)")

# Regra 80/20
top_20_pct_customers = segment_summary.sort_values('revenue', ascending=False).head(4)
print(f"\n  ✅ Regra 80/20: Top 4 segmentos = {top_20_pct_customers['revenue_pct'].sum():.1f}% da receita")
print(f"     (Esperado: 70-80%)")

# Coletar métricas finais
total_customers = len(df_rfm)
total_revenue = df_rfm['monetary'].sum()

print(f"\n📊 MÉTRICAS GLOBAIS:")
print(f"   • Clientes segmentados: {total_customers:,}")
print(f"   • Receita total: R$ {total_revenue:,.2f}")
print(f"   • Ticket médio: R$ {df_rfm['monetary'].mean():.2f}")
print(f"   • Taxa de recompra: {(df_rfm['frequency'] >= 2).sum()/total_customers*100:.1f}%")

print(f"\n💎 SEGMENTOS DE ALTO VALOR:")
high_value_segments = ['Champions', 'Loyal Customers', 'Cannot Lose Them']
high_value_customers = df_rfm[df_rfm['segment'].isin(high_value_segments)]
print(f"   • Clientes: {len(high_value_customers):,} ({(len(high_value_customers)/total_customers*100):.1f}%)")
print(f"   • Receita: R$ {high_value_customers['monetary'].sum():,.2f} ({(high_value_customers['monetary'].sum()/total_revenue*100):.1f}%)")
print(f"   • LTV médio: R$ {high_value_customers['monetary'].mean():.2f}")

print(f"\n⚠️  SEGMENTOS EM RISCO:")
risk_segments = ['At Risk', 'Cannot Lose Them', 'About To Sleep']
risk_customers = df_rfm[df_rfm['segment'].isin(risk_segments)]
print(f"   • Clientes: {len(risk_customers):,} ({(len(risk_customers)/total_customers*100):.1f}%)")
print(f"   • Receita em risco: R$ {risk_customers['monetary'].sum():,.2f} ({(risk_customers['monetary'].sum()/total_revenue*100):.1f}%)")

print(f"\n📈 OPORTUNIDADES DE CRESCIMENTO:")
growth_segments = ['New Customers', 'Potential Loyalist', 'Promising']
growth_customers = df_rfm[df_rfm['segment'].isin(growth_segments)]
print(f"   • Clientes: {len(growth_customers):,} ({(len(growth_customers)/total_customers*100):.1f}%)")
print(f"   • Potencial conversão: +R$ {(growth_customers['monetary'].sum() * 0.5):,.2f} (estimado)")

print(f"\n" + "="*80)
print("💡 PLANO DE AÇÃO POR SEGMENTO")
print("="*80)

# Recomendações por Segmento
for segment in segment_summary.index:
    data = segment_summary.loc[segment]
    print(f"\n🎯 {segment.upper()}:")
    print(f"   • Clientes: {data['customers']:,} ({data['customer_pct']:.1f}%)")
    print(f"   • Receita: R$ {data['revenue']:,.2f} ({data['revenue_pct']:.1f}%)")
    print(f"   • Ação: {ACTION_MAP.get(segment, 'Analisar individualmente')}")
    print(f"   • Prioridade: {PRIORITY_MAP.get(segment, 5)}/6")

print(f"\n" + "="*80)
print("✅ SEGMENTAÇÃO RFM CONCLUÍDA")
print("="*80)